### Note: 
Make sure you go through A0_Initialization first before running the following file.

In [5]:
import math
import time
import os
import cv2 
import keyboard
import pickle
import numpy as np
from PIL import Image, ImageOps, ImageGrab
from matplotlib import pyplot as plt
from skimage import io, color, measure, morphology
from scipy import ndimage
from scipy.ndimage import rotate
import copy
%matplotlib inline

In [6]:
# Load the player_circle_center from the file.
with open('player_circle_center_2.pkl', 'rb') as file:
    player_circle_center = pickle.load(file)
with open('sscoords.pkl', 'rb') as file:
    sscoords = pickle.load(file)
with open('chatcoords.pkl', 'rb') as file:
    chatcoords = pickle.load(file)

furthest_point_distance = np.sqrt((player_circle_center[0])**2 + (player_circle_center[1])**2) # Distance from player to corner of screen. This will be used to measure how far away enemies and items are.

In [8]:
def waitFor(desired_duration):
    start_time = time.perf_counter()
    while True:
        current_time = time.perf_counter()
        if current_time - start_time >= desired_duration:
            break

In [5]:
def calculate_distance(x1, y1, x2, y2):
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance

In [6]:
def GetHotOrCold(original_img, last_iitem_distance): # Returns - 0: Cold, 1: Hot
    posterized_image = ImageOps.posterize(original_img, 2)

    # plt.imshow(posterized_image)
    # plt.show()
    
    image_item_np = np.array(posterized_image)

    # Sizing Img Down
    scale_fraction = 0.2
    width = int(image_item_np.shape[1] * scale_fraction)
    height = int(image_item_np.shape[0] * scale_fraction)
    dim = (width, height)
    image_item_np = cv2.resize(image_item_np, dim)

    # Highlighting the yellow pixels.
    yellow_condition = (image_item_np[:, :, 0] > 150) & (image_item_np[:, :, 1] > 150) & (image_item_np[:, :, 2] < 15) & (abs(image_item_np[:, :, 0] - image_item_np[:, :, 1]) < 15)
    image_item_np[yellow_condition] = [255, 255, 255]
    image_item_np[~yellow_condition] = [0, 0, 0]

    # plt.imshow(image_item_np)
    # plt.show()
    
    # Removing small bits.
    deletion_range = (7, 7)
    size_threshold = 7 # The number of active pixels that have to be within the range around an active pixel.
    
    # Get the coordinates of the white pixels
    white_pixels_mask = (image_item_np[:, :, 0] == 255) & (image_item_np[:, :, 1] == 255) & (image_item_np[:, :, 2] == 255)
    white_pixel_coords = np.argwhere(white_pixels_mask)

    # Iterate over each white pixel's coordinates and removing lone pixels.
    for x, y in white_pixel_coords:
        # Count the number of white pixels in the surrounding area
        active_neighbors = np.sum((abs(white_pixel_coords[:, 0] - x) < deletion_range[0]) & (abs(white_pixel_coords[:, 1] - y) < deletion_range[1]))
        # If the number of white neighbors is less than the minimum required, turn the pixel black
        if active_neighbors < size_threshold:
            image_item_np[x, y] = (0, 0, 0)  # Turn the pixel black
    
    image_item = cv2.cvtColor(image_item_np, cv2.COLOR_BGR2GRAY)

    # Check if the image is already in binary form, if not, apply a threshold
    _, binary_image = cv2.threshold(image_item, 100, 255, cv2.THRESH_BINARY)

    kernel = np.ones((5, 5), np.uint8)
    binary_image = cv2.dilate(binary_image, kernel, iterations=1)

    # Label the objects in the binary image
    labeled_mask, _ = ndimage.label(binary_image)
    
    # Find the properties of the labeled regions
    items_regions = measure.regionprops(labeled_mask)

    closest_item_dist = np.inf
    
    # print(player_circle_center)
    player_zone = int(width/17) # The area around the player to be ignored since it will have the loot carried by the player.
    
    for region in items_regions:
        centroid = region.centroid
        dist = calculate_distance(player_circle_center[0], player_circle_center[1], centroid[0], centroid[1])
        if dist > player_zone:
            if dist < closest_item_dist:
                closest_item_dist = dist
        print("{} - {}".format(centroid, dist))
    
    return (closest_item_dist <= last_iitem_distance), closest_item_dist

In [2]:
from gtts import gTTS
import uuid
from playsound import playsound
import pyautogui

def text_to_speech(text):
    # Create the gTTS object
    tts = gTTS(text=text, lang='en')
    filename = "output_{}.mp3".format(str(uuid.uuid1()))

    # Save the audio file.
    tts.save(filename)

    # Click mouse (For walkie talkie)
    pyautogui.mouseDown()
    # Play the audio file.
    playsound(filename)
    pyautogui.mouseUp()
    
    # Remove audio file.
    os.remove(filename)

### Instructions:
    1- In game, place the terminal parallel to the charging coil so that the charging coil is to your right when you're on the terminal.
    2- Pick a walkie and turn it on.
    3- Enter the terminal.
    4- Run the following cell.

In [126]:
lastDist = np.inf

while True:
    img = ImageGrab.grab(bbox=sscoords)
    waitFor(0.1)
    isHot, lastDist = GetHotOrCold(img, lastDist)
    msg = "No Loot"
    if lastDist != np.inf:
        if isHot:
            msg = "Hot"
        else:
            msg = "Cold"
    print(msg, lastDist)
    text_to_speech(msg)
    waitFor(3)


(59.76744186046512, 80.3953488372093) - 8.703175725692143
(66.2948717948718, 98.43589743589743) - 10.685241777679561
Hot 10.685241777679561
(88.35779816513761, 105.40366972477064) - 29.936433510732023
(94.3855421686747, 123.79518072289157) - 46.952914029576235
Cold 29.936433510732023
(64.14942528735632, 149.816091954023) - 61.81627255330693
(86.20224719101124, 84.15730337078652) - 22.532334493232945
Hot 22.532334493232945
(9.131578947368421, 128.02631578947367) - 67.91648978331797
(18.958904109589042, 119.95890410958904) - 55.22745577061379
(28.948979591836736, 145.48979591836735) - 67.33238943026606
(37.675, 107.9125) - 33.007776072465106
(61.54913294797688, 114.6242774566474) - 26.736845352345437
(72.79569892473118, 99.33333333333333) - 14.346036526475237
(73.26315789473684, 168.26315789473685) - 80.79591950970224
Hot 14.346036526475237
(35.26373626373626, 79.42857142857143) - 29.987368028475064
(53.27710843373494, 97.55421686746988) - 14.361875347339716
(62.35616438356164, 60.0) - 2

KeyboardInterrupt: 